In [17]:
import pandas as pd
import requests
from pandas.io.json import json_normalize
import numpy as np

In [18]:
df = pd.read_csv('listWithAddresses.csv')

In [19]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head()

,Province,City,District / County,Post Code,Address,Latitude,Longitude
0,Anhui,Hefei,Yaohai,230000,"Yaohai,Hefei,Anhui,China",31.860161,117.303740
1,Anhui,Hefei,Baohe,230000,"Baohe,Hefei,Anhui,China",31.795529,117.305249
2,Anhui,Hefei,Feidong,230000,"Feidong,Hefei,Anhui,China",31.890073,117.463606
3,Anhui,Hefei,Luyang,230000,"Luyang,Hefei,Anhui,China",31.880551,117.259237
4,Anhui,Hefei,Changfeng,231100,"Changfeng,Hefei,Anhui,China",32.482035,117.160669


In [20]:
# lets remove those districts for which we dont have the coordinates
df = df[df['Latitude']!='']
df.head()

,Province,City,District / County,Post Code,Address,Latitude,Longitude
0,Anhui,Hefei,Yaohai,230000,"Yaohai,Hefei,Anhui,China",31.860161,117.303740
1,Anhui,Hefei,Baohe,230000,"Baohe,Hefei,Anhui,China",31.795529,117.305249
2,Anhui,Hefei,Feidong,230000,"Feidong,Hefei,Anhui,China",31.890073,117.463606
3,Anhui,Hefei,Luyang,230000,"Luyang,Hefei,Anhui,China",31.880551,117.259237
4,Anhui,Hefei,Changfeng,231100,"Changfeng,Hefei,Anhui,China",32.482035,117.160669


In [21]:
df =df[df['City']=='Shanghai']
df.head()

,Province,City,District / County,Post Code,Address,Latitude,Longitude
2105,Shanghai,Shanghai,Huangpu,200000,"Huangpu,Shanghai,China",31.233593,121.479864
2106,Shanghai,Shanghai,Xuhui,200000,"Xuhui,Shanghai,China",31.163698,121.427994
2107,Shanghai,Shanghai,Jing'an,200000,"Jing'an,Shanghai,China",31.229776,121.443060
2108,Shanghai,Shanghai,Zhabei,200000,"Zhabei,Shanghai,China",31.272406,121.455084
2109,Shanghai,Shanghai,Yangpu,200000,"Yangpu,Shanghai,China",31.262011,121.521430


In [22]:
# lets try to find the venues that belong to a neighbourhood and see what will happen
# lets connect to forsquare
CLIENT_ID = 'EQ31YZNGYY1D22K5N5HWYF1ZBBWAGXKSHW4CZWVUG2MO3K2Y'
CLIENT_SECRET = 'JBYT3Q5NK1PRTYMJPQPDFR5GVLWJF34YCRR1WN3KXUXQN1KB'
VERSION = 20180604
LIMIT = 49
radius = 500
print('Your credentials: ')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+ CLIENT_SECRET)

Your credentials: 
CLIENT_ID: EQ31YZNGYY1D22K5N5HWYF1ZBBWAGXKSHW4CZWVUG2MO3K2Y
CLIENT_SECRET: JBYT3Q5NK1PRTYMJPQPDFR5GVLWJF34YCRR1WN3KXUXQN1KB


In [23]:
#lets write a function to get all venues in one neghibourhood
def getNearbyVenuesByDistance(latitude,longitude,radius,query):
    try:
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude, radius, LIMIT,query)
        results = requests.get(url).json()
        venues = results['response']['groups'][0]['items']
        nearby_venues = json_normalize(venues)
        filteredColumns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng','venue.location.address']
        nearby_venues = nearby_venues[filteredColumns]
        nearby_venues['venue.categories']=nearby_venues['venue.categories'].apply(lambda x: x[0]['name'])
        nearby_venues.rename(columns={'venue.categories': 'Category','venue.name':'Name','venue.location.lat':'Latitude','venue.location.lng':'Longitude','venue.location.address':'address'},inplace=True)
        
        return nearby_venues
    except:
        return False



In [24]:
def getNearbyVenues(latitude,longitude,MinRadius,MaxRadius,trials,query):
    try:
        nearby_venues = pd.DataFrame(columns=['Name','Category','Latitude','Longitude','address'])
        for R in np.linspace(MinRadius,MaxRadius,trials):
            df = getNearbyVenuesByDistance(latitude=latitude,longitude=longitude,radius=R,query=query)
            if type(df)!=bool:
                nearby_venues = nearby_venues.merge(df,how='outer')
        nearby_venues['Latitude'] =  nearby_venues['Latitude'].astype('float64')
        nearby_venues['Longitude'] =  nearby_venues['Longitude'].astype('float64')
        return nearby_venues
    except:
        return False


In [16]:
columns = ['Province','City','District / County','Name','Category','Latitude','Longitude','address']
allVenues  = pd.DataFrame(columns =columns)
allVenues['Latitude'] = allVenues['Latitude'].astype('float64')
allVenues['Longitude'] =  allVenues['Longitude'].astype('float64')

#allVenues['District / County'] = allVenues['District / County'].apply('str')
index = -1
print(df.shape[0])
while index < df.shape[0]-1: 
    index = index + 1
    print('index: ',index)
    City = df.iloc[index]['City']
    print(City)
    allVenues.to_csv('RestaurantListShanghai.csv')
    for ind,r in df.iloc[index:].iterrows():
        if r['City']==City:
            allVenuesNew = getNearbyVenues(latitude=r['Latitude'],longitude=r['Longitude'],MinRadius=500,MaxRadius=500000,trials=50,query='Restaurant')
            allVenuesNew['Province'] =  r['Province']
            allVenuesNew['City'] = r['City']
            allVenuesNew['District / County'] = str(r['District / County'])
            #print('allVenuesNew')
            #print(allVenuesNew.dtypes)
            #print('--------------------------------------------------------')
            #print('allVenues')
            #print(allVenues.dtypes)
            allVenues = allVenues.merge(allVenuesNew,how='outer')
            index = ind
        


17
index:  0
Shanghai


In [13]:
# lets see which categories exist and which are more popular
categories = pd.get_dummies(allVenues['Category'])
categories

,Afghan Restaurant,American Restaurant,Anhui Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,...,Tibetan Restaurant,Truck Stop,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint,Xinjiang Restaurant,Yunnan Restaurant,Zhejiang Restaurant
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
#lets also take a look in cities 
cities = pd.get_dummies(allVenues['City'])
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(cities.sum().sort_values(ascending=False))

Beijing        1001
Guangzhou       671
Chengdu         570
Shenzhen        562
Chongqing       381
Xiamen          347
Qingdao         255
Shenyang        202
Hefei           194
Dalian          193
Fuzhou          176
Zhuhai          142
Yantai          141
Quanzhou        136
Wuhu            133
Weifang         123
Foshan          119
Jinan           117
Jiangmen        113
Huangshan       108
Taizhou         108
Lanzhou          91
Macau            82
Xining           79
Yinchuan         76
Zhangzhou        75
Jieyang          72
Nanning          67
Bengbu           66
Mianyang         63
Liuzhou          61
Anshan           60
Shantou          59
Yingkou          58
Dandong          56
Zhaoqing         56
Guilin           56
Deyang           55
Meizhou          55
Xuancheng        54
Zhanjiang        52
Shangrao         49
Huizhou          47
Qingyuan         46
Zhoushan         46
Maanshan         46
Huaibei          45
Dongguan         44
Putian           44
Nanchong         43


In [39]:
# lets remove the cities for which we have less than a cutoff data
cutoff = 150
citiesAfterCutoff = [i[0].replace(u'\xa0','') for i in cities.sum().sort_values(ascending=False).iteritems() if i[1]>cutoff]
citiesAfterCutoff

['Beijing',
 'Guangzhou',
 'Chengdu',
 'Shenzhen',
 'Chongqing',
 'Xiamen',
 'Qingdao',
 'Shenyang',
 'Hefei',
 'Dalian',
 'Fuzhou']

In [51]:
allVenuesAfterCutoff = allVenues[allVenues['City'].isin(citiesAfterCutoff)].reset_index(drop=True)
allVenuesAfterCutoff

,Name,Category,Latitude,Longitude,address,Province,City,District / County
0,如一坊豆捞,Chinese Restaurant,31.858998,117.303735,铜陵南路,Anhui,Hefei,Yaohai
1,咱姥家大锅台(包河万达公馆店),Anhui Restaurant,31.859937,117.303464,NaN,Anhui,Hefei,Yaohai
2,浏家二叔 LiuYang Steamed Dishes,Hunan Restaurant,31.858226,117.303422,NaN,Anhui,Hefei,Yaohai
3,scirocco all day dining,Buffet,31.862698,117.303426,NaN,Anhui,Hefei,Yaohai
4,Ajisen Ramen (味千拉面),Ramen Restaurant,31.859808,117.297485,马鞍山路130号万达广场1-05号,Anhui,Hefei,Yaohai
5,Mango Thai,Thai Restaurant,31.859727,117.297447,NaN,Anhui,Hefei,Yaohai
6,McDonald's (麦当劳),Fast Food Restaurant,31.859604,117.297442,包河区万达广场一楼,Anhui,Hefei,Yaohai
7,汉拿山韩式烤肉,Korean Restaurant,31.858941,117.297247,马鞍山路万达广场三楼,Anhui,Hefei,Yaohai
8,同庆楼,Chinese Restaurant,31.858715,117.296369,马鞍山路130号万达广场5楼,Anhui,Hefei,Yaohai
9,彩蝶轩 Caidie Bakery,Bakery,31.867661,117.290718,逍遥津站,Anhui,Hefei,Yaohai


In [56]:
# now lets sea what are top 10 food category in each city
# first we should get dummies
#allVenuesAfterCutoff = allVenuesAfterCutoff.join(pd.get_dummies(allVenuesAfterCutoff['Category']))
for city in citiesAfterCutoff:
    print(pd.get_dummies(allVenuesAfterCutoff['Category'][allVenuesAfterCutoff['City']==city]).mean().sort_values(ascending=False)[:10])


Chinese Restaurant      0.212787
Fast Food Restaurant    0.123876
Café                    0.084915
Hotpot Restaurant       0.040959
Asian Restaurant        0.039960
Beijing Restaurant      0.031968
Japanese Restaurant     0.031968
Pizza Place             0.031968
Dumpling Restaurant     0.025974
Italian Restaurant      0.025974
dtype: float64
Chinese Restaurant           0.162444
Fast Food Restaurant         0.122206
Turkish Restaurant           0.064083
Café                         0.058122
Cantonese Restaurant         0.049180
Middle Eastern Restaurant    0.038748
Restaurant                   0.037258
Asian Restaurant             0.034277
Pizza Place                  0.029806
Japanese Restaurant          0.029806
dtype: float64
Chinese Restaurant      0.222807
Fast Food Restaurant    0.149123
Szechuan Restaurant     0.082456
Café                    0.064912
Asian Restaurant        0.061404
Noodle House            0.043860
Hotpot Restaurant       0.042105
BBQ Joint               0.028

In [55]:
categories.mean()

Afghan Restaurant        0.000513
American Restaurant      0.005894
Anhui Restaurant         0.001538
Arepa Restaurant         0.000513
Asian Restaurant         0.037673
                           ...   
Vietnamese Restaurant    0.005638
Wings Joint              0.000256
Xinjiang Restaurant      0.004357
Yunnan Restaurant        0.006663
Zhejiang Restaurant      0.004357
Length: 107, dtype: float64